In [ ]:
# Dependencies

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import random
import hvplot.pandas
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import ast
import numpy as np

In [ ]:
# Reading Congress csv

transactions_df = pd.read_csv("Congress Transactions 2021-2022.csv")

In [ ]:
# Create a list of congress people on the csv
representatives_list = transactions_df.representative.values.tolist()

# Remove duplicates from the list
reps_no_dups = [*set(representatives_list)]

# Choose number of random reps from the list - here we are choosing 6  
random_reps = random.sample(reps_no_dups, 6)

In [ ]:
rand_reps_transactions = transactions_df[transactions_df['representative'].isin(random_reps)]

In [ ]:
# Set type of transaction to purchase only

rand_reps_transactions = transactions_df[transactions_df['representative'].isin(random_reps)]
rand_reps_purchases = rand_reps_transactions[rand_reps_transactions["type"]=="purchase"]
rand_reps_purchases.head()

In [ ]:
# Random Representative and Party

rand_rep_party = rand_reps_transactions[["representative","party"]]
rand_rep_party = rand_rep_party.drop_duplicates().reset_index()
rand_rep_party_index = rand_rep_party.set_index(["representative"])["party"]
pd.DataFrame(rand_rep_party_index)

In [ ]:
# Random Representative % of Party

party_counts = rand_rep_party_index.value_counts()

party_counts.plot.pie(autopct='%.1f%%')
plt.title('Party Distribution of Representatives')
plt.show()

In [ ]:
# Bar Chart for Number of Stock Purchases per Representative

number_purchases= rand_reps_purchases.groupby(["representative"]).count()["type"]
number_purchases.plot(kind="bar")
plt.xlabel("Representative")
plt.ylabel("Number of Purchases")
plt.title("Number of Stock Purchases by Representative")
plt.show()

In [ ]:
# Sectors purchased by random representatives

top_sectors= rand_reps_purchases.groupby(["sector"]).count()["representative"]
top_sectors.plot(kind="bar")
plt.xlabel("Sector")
plt.ylabel("Total")
plt.title("Sectors Purchased by Representative")
plt.show()

In [ ]:
# Representative and Industry

top_sectors= rand_reps_purchases.groupby(["industry"]).count()["representative"]
top_sectors.plot(kind="bar")
plt.xlabel("Industry")
plt.ylabel("Total")
plt.title("Industries Purchased by Representative")
plt.show()

In [ ]:
# Top seven tickers purchased by all representatives

tickers_purchased = rand_reps_purchases.groupby("ticker").size().reset_index(name='counts')
tickers_purchased = tickers_purchased.sort_values("counts", ascending=False).head(7)
tickers_purchased.plot(kind='bar', x='ticker', y='counts', color='skyblue')
plt.xlabel('Ticker')
plt.ylabel('Counts')
plt.title('Number of Purchases by Ticker')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Representative state to be mapped in hvplot

rep_state = rand_reps_transactions[["representative","state"]]
rep_state = rep_state.drop_duplicates().reset_index()
rep_state_index = rep_state.set_index(["representative"])["state"]
pd.DataFrame(rep_state)

In [ ]:
# Adding lat and long to map in hvplot

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

rep_state['latitude'] = None
rep_state['longitude'] = None

for i, row in rep_state.iterrows():
    location = geolocator.geocode(row["state"] + ', United States')
    rep_state.at[i, 'latitude'] = location.latitude
    rep_state.at[i, 'longitude'] = location.longitude

rep_state

In [ ]:
merged_df = pd.merge(rep_state, number_purchases, on='representative')
merged_df

In [ ]:
# Plot representative state and number of purchases on US map

rep_map = merged_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["representative","type"]
)

# Display the map plot
rep_map

In [ ]:
# Get tickers from all transactions for each of the random congress people chosen above
rand_reps_transactions = transactions_df[transactions_df['representative'].isin(random_reps)]
# Set type of transaction to purchase only
rand_reps_purchases = rand_reps_transactions[rand_reps_transactions["type"]=="purchase"]
rand_reps_purchases

In [ ]:
#get the tickers for all transactions in the dataframe of transactions for randomly chosen congress people
tickers = rand_reps_purchases.ticker.values.tolist()
#remove duplicates from tickers
tickers = [*set(tickers)]
tickers

In [ ]:
#tickers to yfinance to get history to get history for specified date range
stock_info = yf.download(tickers, start = "2021-1-1", end = "2022-1-1", ignore_tz=True)
stock_info

In [ ]:
#Copy stock history to a static dataframe so we are not requesting data every time we reference 
stock_static = stock_info["Adj Close"].copy()

#Changing date from index to a column
stock_static.reset_index(inplace=True)
stock_static = stock_static.dropna(axis='columns', how='all')
stock_static

#Have to use this format to access columns
#a = stock_static["Adj Close"]["ACN"].values.tolist()
#a

In [ ]:
# Create a new DataFrame to hold the stock returns for each stock purchased by the random representative
returns_df = pd.DataFrame()

In [ ]:
# Calculate the stock returns for each stock purchased by the random representative
for ticker in tickers:
    returns_df[ticker] = stock_static[ticker].pct_change()

In [ ]:
# Drop any rows with missing values from the returns DataFrame
returns_df.dropna(inplace=True)

In [ ]:
# Fit an regression model to the stock returns
model = sm.OLS(returns_df[tickers[1]], sm.add_constant(returns_df[tickers[0]])).fit()
# Print the coefficient and intercept of the regression model
print(random_rep)
print(tickers[0])
print(tickers[1])
print("Coefficient: ", model.params[1])
print("Intercept: ", model.params[0])
# Get the p-value for the regression
print("P-value: ", model.pvalues[1])

In [ ]:
x = returns_df[tickers[0]].values.reshape(-1, 1)
y = returns_df[tickers[1]].values.reshape(-1, 1)
plt.scatter(x, y, color='blue')
plt.plot(x, reg.predict(x), color='red')
plt.xlabel("Ticker (0) (x)")
plt.ylabel("Ticker (1) (y)")
plt.title("Linear Regression of Stock Purchases")
plt.show()

In [ ]:
#Convert the date format to match the Congress dataframe date format
#stock_static = pd.to_datetime(stock_static["Date"], format = '%Y-%m-%d %H:%M:%S').strftime('%m/%d/%Y')
stock_static["New Date"]= stock_static["Date"].dt.strftime('%#m/%d/%Y')
#dff = dff["Old Time"].dt.strftime("%d/%m/%Y %H:%M")
stock_static

In [ ]:
type(stock_static["New Date"][0])

In [ ]:
acn_purchases = rand_reps_purchases.loc[rand_reps_purchases['ticker'] == "ABT"]
acn_purchases

In [ ]:
# Getting the price data for a specific stock and date
# Purchase_price = stock_static[stock_static["New Date"] =="1/04/2021"]["Adj Close"]["NTAP"]
# Purchase_price

ticker = stock_static["ABT"]
date = stock_static["New Date"]

plt.plot(date, ticker, color='blue', marker='.', linestyle='solid')
plt.axvline("8/03/2021", color = "green")
plt.axvline("5/24/2021", color ="red")


#purchase_price = stock_static[stock_static["New Date"] == "01/04/2021"]["Adj Close"]["AMZN"]


#purchase_date = "01/04/2021"
#purchase_price = stock_static[stock_static["New Date"] == purchase_date]["Adj Close"]["AMZN"]

#plt.plot(purchase_date,purchase_price, marker = "o",color="red")
#transaction = acn_purchases["transaction_date"]


#x = stock_static["Adj Close"]["ABNB"] & date == transaction
#print(x)
#plt.plot(transaction)
#months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan']
#plt.xticks(np.linspace(0,365,13), months)



#"= df['date'].apply(pd.Timestamp)
#df=df.set_index('date')
#df.plot(marker='o')

plt.show()

In [ ]:
transactions_df['ticker'].value_counts().head(20)

In [ ]:
# create a dictionary to store the data
data = {}

# get the unique representative names
unique_reps = transactions_df['representative'].unique()

# loop through each representative
for rep in unique_reps:
  # get the data for the current representative
  rep_data = transactions_df[transactions_df['representative'] == rep]
  # get the stock ticker they traded the most
  most_traded_ticker = rep_data['ticker'].value_counts().index[0]
  #get the total count of ticker trades
  rep_ticker_counts = rep_data['ticker'].count()
  # get the highest value for that ticker
  highest_value = rep_data[rep_data['ticker'] == most_traded_ticker]['amount'].max()
  # get party affiliation  
  party = rep_data['party'].iloc[0] 
  #add trade type
  trade_type = rep_data['type'].iloc[0]
  # add the data to the dictionary
  data[rep] = [party, most_traded_ticker, rep_ticker_counts, trade_type, highest_value]
  

#create a dataframe from the dictionary
new_df = pd.DataFrame.from_dict(data, orient='index', columns=['Party', 'Ticker', 'Total Trades','Trade','Highest Value'])
new_df.index.name = 'Rep Name'
new_df = new_df.sort_values(by='Total Trades', ascending=False)

new_df

In [ ]:
# separate the data into two dataframes: one for Democrats and one for Republicans
democrats_df = new_df[new_df['Party'] == 'Democrat']
republicans_df = new_df[new_df['Party'] == 'Republican']

# take a random sample of 50/50 representatives from each party 
democrats_sample = democrats_df[democrats_df['Total Trades'] >= 10].sample(n=3)
republicans_sample = republicans_df[republicans_df['Total Trades'] >= 10].sample(n=3)

# Finally, concatenate the two samples into one dataframe
random_sample = pd.concat([democrats_sample, republicans_sample])

random_sample

In [ ]:
# Define the stock ticker you want to obtain the history data for
ticker_name = 'MSFT'

# Use the `yf.download` method to obtain the historical data
data = yf.download(ticker_name, start='2021-01-01', end='2022-01-31')

# Convert the data into a Pandas DataFrame
yf_df = pd.DataFrame(data)

# Reset the index to make the date information a column
yf_df = yf_df.reset_index()

# Set the date column as the index for the dataframe
yf_df = yf_df.set_index('Date')

# Filter the dataframe to only keep the "Adj Close" column
yf_df = yf_df[['Adj Close']]

# Calculate the daily percent change for each individual ticker
percent_change = [np.nan]
for i in range(1, len(yf_df)):
    daily_change = ((yf_df.iloc[i]['Adj Close'] - yf_df.iloc[i - 1]['Adj Close']) / yf_df.iloc[i - 1]['Adj Close'] * 100)
    percent_change.append(daily_change)

original_row = pd.DataFrame(percent_change, columns=["percent_change"])

# Set the date information as the index for the original_row dataframe
original_row = original_row.set_index(yf_df.index)

# Aggregate the daily change into weekly
weekly_change = original_row.resample('W').sum()

# Create a DataFrame with all the transaction dates for a single ticker
ticker = 'MSFT'
ticker_dates_df = transactions_df.loc[transactions_df['ticker'] == ticker_name, ['transaction_date']]

# Convert the transaction dates to datetime format
ticker_dates_df['transaction_date'] = pd.to_datetime(ticker_dates_df['transaction_date'])

# Get a list of unique dates
ticker_dates = ticker_dates_df['transaction_date'].unique()

# Plot the weekly change
fig, ax = plt.subplots(figsize=(30,6))
ax.plot_date(weekly_change.index, weekly_change['percent_change'], '-')
ax.set_xlabel('Week')
ax.set_ylabel('Change')
ax.set_title(f"Weekly % Change in {ticker_name} Price")

# Add a marker for each transaction date that falls within the date range of weekly_change
for date in ticker_dates:
    if date >= weekly_change.index.min() and date <= weekly_change.index.max():
        ax.plot_date(date, 0, marker='D', markersize=8, color='red')

plt.show()

In [ ]:
ticker_dates_df.value_counts().head(10)

In [ ]:
# Define the stock ticker you want to obtain the history data for
ticker_name = 'NTAP'

# Use the `yf.download` method to obtain the historical data
data = yf.download(ticker_name, start='2021-01-01', end='2022-01-31')

# Convert the data into a Pandas DataFrame
yf_df = pd.DataFrame(data)

# Reset the index to make the date information a column
yf_df = yf_df.reset_index()

# Set the date column as the index for the dataframe
yf_df = yf_df.set_index('Date')

# Filter the dataframe to only keep the "Adj Close" column
yf_df = yf_df[['Adj Close']]

# Calculate the daily percent change for each individual ticker
percent_change = [np.nan]
for i in range(1, len(yf_df)):
    daily_change = ((yf_df.iloc[i]['Adj Close'] - yf_df.iloc[i - 1]['Adj Close']) / yf_df.iloc[i - 1]['Adj Close'] * 100)
    percent_change.append(daily_change)

original_row = pd.DataFrame(percent_change, columns=["percent_change"])

# Set the date information as the index for the original_row dataframe
original_row = original_row.set_index(yf_df.index)

In [ ]:
# Aggregate the daily change into weekly
weekly_change = original_row.resample('W').sum()

# Create a DataFrame with all the transaction dates for a single ticker
ticker = 'NTAP'
ticker_dates_df = transactions_df.loc[transactions_df['ticker'] == ticker_name, ['transaction_date']]

# Convert the transaction dates to datetime format
ticker_dates_df['transaction_date'] = pd.to_datetime(ticker_dates_df['transaction_date'])

# Get a list of unique dates
ticker_dates = ticker_dates_df['transaction_date'].unique()

# Plot the weekly change
fig, ax = plt.subplots(figsize=(30,6))
ax.plot_date(weekly_change.index, weekly_change['percent_change'], '-')
ax.set_xlabel('Week')
ax.set_ylabel('Change')
ax.set_title(f"Weekly % Change in {ticker_name} Price")

# Add a marker for each transaction date that falls within the date range of weekly_change
for date in ticker_dates:
    if date >= weekly_change.index.min() and date <= weekly_change.index.max():
        ax.plot_date(date, 0, marker='D', markersize=8, color='red')

plt.show()

In [ ]:
ticker_dates_df.value_counts().head(10)

In [ ]:
# Define the stock ticker you want to obtain the history data for
ticker_name = 'TSLA'

# Use the `yf.download` method to obtain the historical data
data = yf.download(ticker_name, start='2021-01-01', end='2022-01-31')

# Convert the data into a Pandas DataFrame
yf_df = pd.DataFrame(data)

# Reset the index to make the date information a column
yf_df = yf_df.reset_index()

# Set the date column as the index for the dataframe
yf_df = yf_df.set_index('Date')

# Filter the dataframe to only keep the "Adj Close" column
yf_df = yf_df[['Adj Close']]

# Calculate the daily percent change for each individual ticker
percent_change = [np.nan]
for i in range(1, len(yf_df)):
    daily_change = ((yf_df.iloc[i]['Adj Close'] - yf_df.iloc[i - 1]['Adj Close']) / yf_df.iloc[i - 1]['Adj Close'] * 100)
    percent_change.append(daily_change)

original_row = pd.DataFrame(percent_change, columns=["percent_change"])

# Set the date information as the index for the original_row dataframe
original_row = original_row.set_index(yf_df.index)

# Aggregate the daily change into weekly
weekly_change = original_row.resample('W').sum()

# Create a DataFrame with all the transaction dates for a single ticker
ticker = 'TSLA'
ticker_dates_df = transactions_df.loc[transactions_df['ticker'] == ticker_name, ['transaction_date']]

# Convert the transaction dates to datetime format
ticker_dates_df['transaction_date'] = pd.to_datetime(ticker_dates_df['transaction_date'])

# Get a list of unique dates
ticker_dates = ticker_dates_df['transaction_date'].unique()

# Plot the weekly change
fig, ax = plt.subplots(figsize=(30,6))
ax.plot_date(weekly_change.index, weekly_change['percent_change'], '-')
ax.set_xlabel('Week')
ax.set_ylabel('Change')
ax.set_title(f"Weekly % Change in {ticker_name} Price")

# Add a marker for each transaction date that falls within the date range of weekly_change
for date in ticker_dates:
    if date >= weekly_change.index.min() and date <= weekly_change.index.max():
        ax.plot_date(date, 0, marker='D', markersize=8, color='red')

plt.show()

In [ ]:
ticker_dates_df.value_counts().head(10)

In [ ]:
# Define the stock ticker you want to obtain the history data for
ticker_name = 'AAPL'

# Use the `yf.download` method to obtain the historical data
data = yf.download(ticker_name, start='2021-01-01', end='2022-01-31')

# Convert the data into a Pandas DataFrame
yf_df = pd.DataFrame(data)

# Reset the index to make the date information a column
yf_df = yf_df.reset_index()

# Set the date column as the index for the dataframe
yf_df = yf_df.set_index('Date')

# Filter the dataframe to only keep the "Adj Close" column
yf_df = yf_df[['Adj Close']]

# Calculate the daily percent change for each individual ticker
percent_change = [np.nan]
for i in range(1, len(yf_df)):
    daily_change = ((yf_df.iloc[i]['Adj Close'] - yf_df.iloc[i - 1]['Adj Close']) / yf_df.iloc[i - 1]['Adj Close'] * 100)
    percent_change.append(daily_change)

original_row = pd.DataFrame(percent_change, columns=["percent_change"])

# Set the date information as the index for the original_row dataframe
original_row = original_row.set_index(yf_df.index)

In [ ]:
# Aggregate the daily change into weekly
weekly_change = original_row.resample('W').sum()

# Create a DataFrame with all the transaction dates for a single ticker
ticker = 'AAPL'
ticker_dates_df = transactions_df.loc[transactions_df['ticker'] == ticker_name, ['transaction_date']]

# Convert the transaction dates to datetime format
ticker_dates_df['transaction_date'] = pd.to_datetime(ticker_dates_df['transaction_date'])

# Get a list of unique dates
ticker_dates = ticker_dates_df['transaction_date'].unique()

# Plot the weekly change
fig, ax = plt.subplots(figsize=(30,6))
ax.plot_date(weekly_change.index, weekly_change['percent_change'], '-')
ax.set_xlabel('Week')
ax.set_ylabel('Change')
ax.set_title(f"Weekly % Change in {ticker_name} Price")

# Add a marker for each transaction date that falls within the date range of weekly_change
for date in ticker_dates:
    if date >= weekly_change.index.min() and date <= weekly_change.index.max():
        ax.plot_date(date, 0, marker='D', markersize=8, color='red')

plt.show()

In [ ]:
ticker_dates_df.value_counts().head(10)

In [ ]:
# Define the stock ticker you want to obtain the history data for
ticker_name = 'NVDA'

# Use the `yf.download` method to obtain the historical data
data = yf.download(ticker_name, start='2021-01-01', end='2022-01-31')

# Convert the data into a Pandas DataFrame
yf_df = pd.DataFrame(data)

# Reset the index to make the date information a column
yf_df = yf_df.reset_index()

# Set the date column as the index for the dataframe
yf_df = yf_df.set_index('Date')

# Filter the dataframe to only keep the "Adj Close" column
yf_df = yf_df[['Adj Close']]

# Calculate the daily percent change for each individual ticker
percent_change = [np.nan]
for i in range(1, len(yf_df)):
    daily_change = ((yf_df.iloc[i]['Adj Close'] - yf_df.iloc[i - 1]['Adj Close']) / yf_df.iloc[i - 1]['Adj Close'] * 100)
    percent_change.append(daily_change)

original_row = pd.DataFrame(percent_change, columns=["percent_change"])

# Set the date information as the index for the original_row dataframe
original_row = original_row.set_index(yf_df.index)

In [ ]:
# Aggregate the daily change into weekly
weekly_change = original_row.resample('W').sum()

# Create a DataFrame with all the transaction dates for a single ticker
ticker = 'NVDA'
ticker_dates_df = transactions_df.loc[transactions_df['ticker'] == ticker_name, ['transaction_date']]

# Convert the transaction dates to datetime format
ticker_dates_df['transaction_date'] = pd.to_datetime(ticker_dates_df['transaction_date'])

# Get a list of unique dates
ticker_dates = ticker_dates_df['transaction_date'].unique()

# Plot the weekly change
fig, ax = plt.subplots(figsize=(30,6))
ax.plot_date(weekly_change.index, weekly_change['percent_change'], '-')
ax.set_xlabel('Week')
ax.set_ylabel('Change')
ax.set_title(f"Weekly % Change in {ticker_name} Price")

# Add a marker for each transaction date that falls within the date range of weekly_change
for date in ticker_dates:
    if date >= weekly_change.index.min() and date <= weekly_change.index.max():
        ax.plot_date(date, 0, marker='D', markersize=8, color='red')

plt.show()

In [ ]:
ticker_dates_df.value_counts().head(15)